This notebook is an exploration of how to access DPIRD and DBCA data layers via access endpoints.

WA govt offers a service called '[SLIP](https://data.wa.gov.au/slip)' that has two services, and [ESRI ArcGIS REST server](https://services.slip.wa.gov.au/public/rest/services/SLIP_Public_Services/Soil_Landscape/MapServer), and an [OGC WFS (web feature service)](https://services.slip.wa.gov.au/public/services/SLIP_Public_Services/Soil_Landscape_WFS/MapServer/WFSServer?). the WFS will work for vector data.

- [wa dep web API's](https://www.agric.wa.gov.au/web-apis) to explore
- Landgate use slip to distribute spatial data - investigate the [12 services available](https://catalogue.data.wa.gov.au/group/?q=Landgate+SLIP) as some include property street addresses, cadastral services etc.
- SLIP public services - https://catalogue.data.wa.gov.au/group/?q=SLIP+Public

There appear to be 27 'layers' in the DPIRD dataset - should confirm with Terrawise which ones they want access to

TODO: add DPIRD to the python geodata harvester

TODO: owslib WFS handles coordinates differently to geopandas,OWSLib will switch the axis order from EN to NE automatically if designated by EPSG-Registry. Add a utility to the sendand geo python package to handle this conversion as we will be interacting with OGC web services regularly

Notes:

- WFS and WMS don't seem suitable. WMS returns a jpeg or image without spatial data embedded. WFS seems broken.
- ESRI Arc server
- DPIRD API - made a sensand account to get an API token.


https://ad.vgiscience.org/links/posts/2024-01-07-wfs-feature-layer-download/

In [1]:
%pip install --upgrade hvplot holoviews panel jupyter_bokeh bokeh -q
%pip install dask[dataframe] -q
%pip install geojson -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import io
from io import BytesIO
import json
import requests
from urllib import parse, request
import holoviews as hv
import hvplot.pandas
import bokeh
import panel as pn
import geopandas as gpd
from shapely.geometry import box
from owslib.fes import *
from owslib.etree import etree
from owslib.wfs import WebFeatureService
from owslib.wms import WebMapService

import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, interact

from fiona import BytesCollection 
from shapely.geometry import box 
from lxml import etree 

hv.extension("bokeh")
pn.extension()

In [3]:
print('Panel version:', pn.__version__)
print('Bokeh version:', bokeh.__version__)

Panel version: 1.4.4
Bokeh version: 3.4.1


In [4]:
# import area of interest

input_data_dir = "/workspace/notebooks/sandbox/data/input-data/DPIRD-testing"
output_data_dir = "/workspace/notebooks/sandbox/data/output-data/terrawise"

input_aoi = os.path.join(input_data_dir, "dpird-test2.shp")

aoi_gpd = gpd.read_file(input_aoi)

In [5]:
target_crs = 'EPSG:4326'  # WGS 84
DPIRD_crs = 'EPSG:7844' #this is GDA2020

if aoi_gpd.crs != DPIRD_crs:
    aoi_gpd = aoi_gpd.to_crs(DPIRD_crs)
    


bbox = list(aoi_gpd.total_bounds)

#NOTE: WFS handle coordinates differently to standard bounding boxes (yes, it's annoying as hell). 
#Convert a standard bounding box to the correct format before trying to use it in a WFS request.
lat_long_bounds = (bbox[1], bbox[0], bbox[3], bbox[2])

print(f"The original bounding box: {bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}")
print(f"The lat/long bounding box: {lat_long_bounds}")

The original bounding box: 116.6340516546263,-32.185630913089035,117.06139232448008,-31.797209996436898
The lat/long bounding box: (-32.185630913089035, 116.6340516546263, -31.797209996436898, 117.06139232448008)


In [6]:
#set up access to DPIRD WFS endpoint
wfs_resource_url = "https://public-services.slip.wa.gov.au/public/services/SLIP_Public_Services/Soil_Landscape_WFS/MapServer/WFSServer"
wfs = WebFeatureService(url=wfs_resource_url, version='2.0.0')


In [7]:
# TODO: use panel to enable user to select which WFS layer to display

print(f" WFS title: {wfs.identification.title}")
print(f" WFS operations available: {[operation.name for operation in wfs.operations]}")
print(f"Get WFS get feature methods: {wfs.getOperationByName('GetFeature').methods}")
print(f"Get WFS get feature format options: {wfs.getOperationByName('GetFeature').formatOptions}")


 WFS title: WFS
 WFS operations available: ['GetCapabilities', 'DescribeFeatureType', 'GetPropertyValue', 'GetFeature', 'GetGmlObject', 'ListStoredQueries', 'DescribeStoredQueries', 'ImplementsBasicWFS', 'ImplementsTransactionalWFS', 'ImplementsLockingWFS', 'KVPEncoding', 'XMLEncoding', 'SOAPEncoding', 'ImplementsInheritance', 'ImplementsRemoteResolve', 'ImplementsResultPaging', 'ImplementsStandardJoins', 'ImplementsSpatialJoins', 'ImplementsTemporalJoins', 'ImplementsFeatureVersioning', 'ManageStoredQueries', 'CountDefault']
Get WFS get feature methods: [{'constraints': [], 'type': 'Get', 'url': 'https://public-gr-admin.slip.wa.gov.au:443/public/services/SLIP_Public_Services/Soil_Landscape_WFS/MapServer/WFSServer?'}, {'constraints': [], 'type': 'Post', 'url': 'https://public-gr-admin.slip.wa.gov.au:443/public/services/SLIP_Public_Services/Soil_Landscape_WFS/MapServer/WFSServer'}]
Get WFS get feature format options: ['text/xml']


In [13]:
list(wfs.contents)

feature_type = list(wfs.contents.keys())

for feature_type in wfs.contents:
    print(feature_type)

esri:Soil_landscape_land_quality_-_Flood_Risk__DPIRD-007_
esri:Soil_landscape_land_quality_-_Phosphorus_Export_Risk__DPIRD-010_
esri:Soil_landscape_land_quality_-_Salinity_Risk__DPIRD-009_
esri:Soil_landscape_land_quality_-_Subsurface_Acidification_Risk__DPIRD-011_
esri:Soil_landscape_land_quality_-_Subsurface_Compaction_Risk__DPIRD-012_
esri:Soil_landscape_land_quality_-_Water_Erosion_Risk__DPIRD-013_
esri:Soil_landscape_land_quality_-_Water_Repellence_Risk__DPIRD-014_
esri:Soil_landscape_land_quality_-_Waterlogging_Risk__DPIRD-015_
esri:Soil_landscape_land_quality_-_Wind_Erosion_Risk__DPIRD-016_
esri:Soil_landscape_land_quality_-_Zones__DPIRD-017_
esri:Soil_Landscape_Mapping_-_Best_Available__DPIRD-027_
esri:Soil_landscape_land_quality_-_Surface_Acidity__current___DPIRD-035_
esri:Soil_landscape_land_quality_-_Subsurface_Acidity__current___DPIRD-036_
esri:Soil_landscape_land_quality_-_Surface_Alkalinity__current___DPIRD-037_
esri:Soil_landscape_land_quality_-_Subsurface_Alkalinity__cu

In [279]:
response = wfs.getfeature(typename='esri:Soil_Landscape_Mapping_-_Best_Available__DPIRD-027_',
                          bbox=lat_long_bounds)

In [280]:
# Why are we writing our data as an XML? Because for some reason, that's the only output format DPIRD SLIP will let us!
out = open('/workspace/notebooks/sandbox/data/output-data/terrawise/test-storedquery.xml', 'wb')
out.write(response.read())
out.close()

In [281]:
data = gpd.read_file('/workspace/notebooks/sandbox/data/output-data/terrawise/test-storedquery.xml')

data.head()

,gml_id,objectid,id,DPIRD_Project_Code,Soil-landscape_mapping_project_name,Project_reliability,Group_of_projects_to_which_project_belongs,Full_soil-landscape_map_unit_symbol,Unique_map_unit_identifier,Map_unit_name,...,Land_Capability_Code_Perennial_Horticulture,Percent_Class_1_or_2_Perennial_Horticulture,Percent_Class_1__2_or_3_Perennial_Horticulture,Land_Capability_Decode_Perennial_Horticulture,Land_Capability_Code_Vines,Percent_Class_1_or_2_Vines,Percent_Class_1__2_or_3_Vines,Land_Capability_Decode_Vines,st_perimeter_shape_,geometry
0,Soil_Landscape_Mapping_-_Best_Available__DPIRD...,2093,2093.0,NOR,Northam land resources survey,"Medium data quality, midscale or imprecise map...",SWC,253DaKO,6799,Kokeby subsystem,...,A2,59,81,50-70% of the land has high to very high capab...,A2,60,99,50-70% of the land has high to very high capab...,0.026490,"MULTIPOLYGON (((116.66738 -32.13059, 116.66738..."
1,Soil_Landscape_Mapping_-_Best_Available__DPIRD...,2094,2094.0,NOR,Northam land resources survey,"Medium data quality, midscale or imprecise map...",SWC,253DaKO,6799,Kokeby subsystem,...,A2,59,81,50-70% of the land has high to very high capab...,A2,60,99,50-70% of the land has high to very high capab...,0.075891,"MULTIPOLYGON (((116.68009 -32.14731, 116.67998..."
2,Soil_Landscape_Mapping_-_Best_Available__DPIRD...,2095,2095.0,NOR,Northam land resources survey,"Medium data quality, midscale or imprecise map...",SWC,253DaKO,6799,Kokeby subsystem,...,A2,59,81,50-70% of the land has high to very high capab...,A2,60,99,50-70% of the land has high to very high capab...,0.104885,"MULTIPOLYGON (((116.65678 -32.15641, 116.65670..."
3,Soil_Landscape_Mapping_-_Best_Available__DPIRD...,2096,2096.0,NOR,Northam land resources survey,"Medium data quality, midscale or imprecise map...",SWC,253DaKO,6799,Kokeby subsystem,...,A2,59,81,50-70% of the land has high to very high capab...,A2,60,99,50-70% of the land has high to very high capab...,0.104224,"MULTIPOLYGON (((116.75306 -32.17700, 116.75294..."
4,Soil_Landscape_Mapping_-_Best_Available__DPIRD...,2097,2097.0,NOR,Northam land resources survey,"Medium data quality, midscale or imprecise map...",SWC,253DaKO,6799,Kokeby subsystem,...,A2,59,81,50-70% of the land has high to very high capab...,A2,60,99,50-70% of the land has high to very high capab...,0.045905,"MULTIPOLYGON (((116.72012 -32.17875, 116.71966..."


In [282]:

#when clipping the data, use the ORIGINAL bounding box, not the lat/long one
data_clip = data.clip(bbox)
data_clip.to_file((os.path.join(output_data_dir,'test-dpird_clip.geojson')), driver='GeoJSON')

In [289]:
aoi = aoi_gpd.hvplot(geo=True, tiles=True, alpha=0.5, line_color='red', width=1200, height=800)
dpird_test = data_clip.hvplot(geo=True, tiles=True, c="Map_unit_name", alpha=0.8, tiles_opts={'alpha': 0.5}, width=1200, height=800)

combined_map = aoi * dpird_test
pn.panel(combined_map).show()

Launching server at http://localhost:44803
